# KoGPT-2를 활용한 Few-shot & Zero-shot 실습


> 작성자      
```
* 김성현 (bananaband657@gmail.com)  
김바다 (qkek983@gmail.com)
박상희 (parksanghee0103@gmail.com)  
이정우 (jungwoo.l2.rs@gmail.com)
```
[CC BY-NC-ND](https://creativecommons.org/licenses/by-nc-nd/2.0/kr/)

Transformers를 설치하고 Model을 load하겠습니다.

In [1]:
!pip install transformers

     |████████████████████████████████| 2.1MB 13.6MB/s 
     |████████████████████████████████| 3.3MB 53.4MB/s 
     |████████████████████████████████| 901kB 55.1MB/s 


In [2]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/taeminlee/kogpt2

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 79 not upgraded.
Need to get 6,229 kB of archives.
After this operation, 14.5 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu bionic/main amd64 git-lfs amd64 2.13.3 [6,229 kB]
Fetched 6,229 kB in 1s (7,714 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 160987 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.13.3_a

In [3]:
import torch
from tokenizers import SentencePieceBPETokenizer
from transformers import GPT2Config, GPT2LMHeadModel

tokenizer = SentencePieceBPETokenizer("/content/kogpt2/vocab.json", "/content/kogpt2/merges.txt")

config = GPT2Config(vocab_size=50000)
config.pad_token_id = tokenizer.token_to_id('<pad>')
model = GPT2LMHeadModel(config)

model_dir = '/content/kogpt2/pytorch_model.bin'

model.load_state_dict(torch.load(model_dir, map_location='cuda'), strict=False)
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [4]:
tokenized_text = tokenizer.encode('이순신은 조선 중기의 무신이다.', add_special_tokens=True)
print(tokenized_text)
print(tokenized_text.tokens)
print(tokenized_text.ids)

Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['▁이순', '신은', '▁조선', '▁중기의', '▁무신', '이다', '.']
[10925, 6647, 1117, 40249, 39793, 128, 47440]


In [5]:
tokenizer.add_special_tokens(["<s>", "</s>"])

0

In [6]:
import torch
torch.manual_seed(42)

input_ids = torch.tensor(tokenizer.encode("이순신은", add_special_tokens=True).ids).unsqueeze(0).to('cuda')

output_sequences = model.generate(input_ids=input_ids, do_sample=True, max_length=100, num_return_sequences=3)
for generated_sequence in output_sequences:
    generated_sequence = generated_sequence.tolist()
    print("GENERATED SEQUENCE : {0}".format(tokenizer.decode(generated_sequence, skip_special_tokens=True)))

GENERATED SEQUENCE : 이순신은 백옥담(이종원 분)의 시선으로 백성을 살피는 것을 알게 되었고, 천하에 대한 소신을 가지게 되었다. 또한 성읍인 성전은 백옥담의 동생인 윤(尹)이 성전과 함께 있었는데 윤을 그의 아버지라고 불렀다. 또한 성전이 완공될 때까지 성전에서 계속 놀 수 있었다. 성전 안에서는 두 권의 도서가 있고, 이 문서에는 윤과 성국의 성전의 위치가 찍혀져 있다.
GENERATED SEQUENCE : 이순신은 이미 태연에게 마음을 돌린 상태. 이어 "자칫 북한이 미사일 도발에 나설 수도 있다는 경고가 되고 있는 만큼 한미가 이에 대해 적절히 대응해주길 바란다"고 당부했다. 또한 "오늘 김관진 제1위원장이 오랫만에 공개토론을 한 뒤 북한 도발을 중지하고 대화를 시작해야 한다고 강조한 것을 감안할 때 이번 주 미측과 한반도에서의 대화 분위기가 계속될 것으로 확신한다"고 말했다. 또한 김 제1위원장의 이번 방문에서 미국측에 한반도 신뢰프로세스 차원의
GENERATED SEQUENCE : 이순신은 고려 왕조를 건국한 이성계 형 이인임부터 명나라로 이민을 온 고려 왕조에 이르기까지 다양한 인물상과 고려 역사를 사실 그대로 옮겨 적에게 전달하는 것이 중요한 사료다. 이들은 모두 6명이고, 이중 6명은 유랑 생활을 한 이른바 ‘귀족’으로, 유랑 생활 5년 동안 ‘무덤덤’을 팠고, 이들은 유림의 한 사람인 목침을 써서 한방에서 술을 빚었다고 한다. 목침은 조선시대 중종


In [7]:
def get_gpt_output(input_sent):
    input_ids = torch.tensor(tokenizer.encode(input_sent, add_special_tokens=True).ids).unsqueeze(0).to('cuda')
    sample_outputs = model.generate(
        input_ids,
        do_sample=True, 
        max_length=512, 
        top_k=50, 
        top_p=0.95, 
        num_return_sequences=1,
        eos_token_id=tokenizer.token_to_id("</s>"),
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    generated_sequence = sample_outputs[0].tolist()
    return tokenizer.decode(generated_sequence, skip_special_tokens=True)

In [8]:
get_gpt_output("<s>이순신은")

"이순신은 그 길로 서찰독(이순신 분)의 아들 노국강(이천희 분), 강효주(한효주도 분)'의 어머니 백도인(홍혜지 분)."

In [9]:
get_gpt_output("<s>철수 : 영희야 안녕!</s><s>영희 : ")

'철수 : 영희야 안녕! 영희 : , 그건 뭐 어때?'

In [10]:
get_gpt_output("<s>철수 : 영희야 안녕!</s><s>영희 : 어! 철수야! 오랜만이다!</s><s>철수 : 그러게~ 잘 지냈어?</s><s>영희 : ")

'철수 : 영희야 안녕! 영희 : 어! 철수야! 오랜만이다! 철수 : 그러게~ 잘 지냈어? 영희 : ^^* 오늘 너무 너무 고마웠어요 그제서야 오늘 아침에 너무 감사했어요 힝^'

In [11]:
get_gpt_output("<s>본문 : 아.. 기분 진짜 짜증나네ㅡㅡ</s><s>감정 : 분노</s><s>본문 : 와!! 진짜 너무 좋아!!</s><s>감정 : ")

'본문 : 아.. 기분 진짜 짜증나네 감정 : 분노 본문 : 와!! 진짜 너무 좋아!! 감정 : ^^* -*(*)*-<(^0^)^$^....'

In [12]:
get_gpt_output("<s>질문 : 코로나 바이러스에 걸리면 어떻게 되나요?</s>\
<s>답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다.</s>\
<s>질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까?</s>\
<s>답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다.</s>\
<s>질문 : 딸기 식물의 수명주기는 무엇입니까?</s>\
<s>답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다.</s>\
<s>질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까?</s>\
<s>답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다.</s>\
<s>질문 : 뇌의 어떤 부분이 말을 제어합니까?</s>\
<s>답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다.</s>\
<s>질문 : 인공지능의 미래에 대해 어떻게 생각하십니까?</s>\
<s>답 : ")

'질문 : 코로나 바이러스에 걸리면 어떻게 되나요? 답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다. 질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까? 답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다. 질문 : 딸기 식물의 수명주기는 무엇입니까? 답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다. 질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까? 답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다. 질문 : 뇌의 어떤 부분이 말을 제어합니까? 답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다. 질문 : 인공지능의 미래에 대해 어떻게 생각하십니까? 답 : 햅틱스 기술은 인간의 정신력에서 결정합니다.: 지능이 인간의 유전자에 의해 조작되기 시작할 때 - 더 나아가서는 컴퓨터로 정보를 전송하는 ^35. .'

In [13]:
get_gpt_output("<s>질문 : 코로나 바이러스에 걸리면 어떻게 되나요?</s>\
<s>답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다.</s>\
<s>질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까?</s>\
<s>답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다.</s>\
<s>질문 : 딸기 식물의 수명주기는 무엇입니까?</s>\
<s>답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다.</s>\
<s>질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까?</s>\
<s>답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다.</s>\
<s>질문 : 뇌의 어떤 부분이 말을 제어합니까?</s>\
<s>답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다.</s>\
<s>질문 : 자연어처리에서 언어모델이란 무엇입니까?</s>\
<s>답 : ")

'질문 : 코로나 바이러스에 걸리면 어떻게 되나요? 답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다. 질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까? 답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다. 질문 : 딸기 식물의 수명주기는 무엇입니까? 답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다. 질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까? 답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다. 질문 : 뇌의 어떤 부분이 말을 제어합니까? 답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다. 질문 : 자연어처리에서 언어모델이란 무엇입니까? 답 : appromotion의 초기 단계에서 인간의 의사소통의 능력을 평가하는 것으로 표현됩니다, 또는 언어영역에서는 언어과 언어를 다루는 데 쓰이는 여러 가지 기법을 사용합니다. ----------- aptive Abouting The free: The speech Abit. (c) ac - afety Ecceptor by Deligion. to all-the acconformation in excess and datas of organization and recent gradual mania) I

In [14]:
get_gpt_output("<s>질문 : 코로나 바이러스에 걸리면 어떻게 되나요?</s>\
<s>답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다.</s>\
<s>질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까?</s>\
<s>답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다.</s>\
<s>질문 : 딸기 식물의 수명주기는 무엇입니까?</s>\
<s>답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다.</s>\
<s>질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까?</s>\
<s>답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다.</s>\
<s>질문 : 뇌의 어떤 부분이 말을 제어합니까?</s>\
<s>답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다.</s>\
<s>질문 : 대한민국에서 최고의 인공지능 기술을 보유한 기업은 어디입니까?</s>\
<s>답 : ")

'질문 : 코로나 바이러스에 걸리면 어떻게 되나요? 답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다. 질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까? 답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다. 질문 : 딸기 식물의 수명주기는 무엇입니까? 답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다. 질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까? 답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다. 질문 : 뇌의 어떤 부분이 말을 제어합니까? 답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다. 질문 : 대한민국에서 최고의 인공지능 기술을 보유한 기업은 어디입니까? 답 : 렉스엘렉스(Leieso C.L)라고 합니다. ...'

In [15]:
get_gpt_output("<s>질문 : 코로나 바이러스에 걸리면 어떻게 되나요?</s>\
<s>답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다.</s>\
<s>질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까?</s>\
<s>답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다.</s>\
<s>질문 : 딸기 식물의 수명주기는 무엇입니까?</s>\
<s>답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다.</s>\
<s>질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까?</s>\
<s>답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다.</s>\
<s>질문 : 뇌의 어떤 부분이 말을 제어합니까?</s>\
<s>답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다.</s>\
<s>질문 : 이순신 장군이 전사한 전투는 무슨 전투입니까?</s>\
<s>답 : ")

'질문 : 코로나 바이러스에 걸리면 어떻게 되나요? 답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다. 질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까? 답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다. 질문 : 딸기 식물의 수명주기는 무엇입니까? 답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다. 질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까? 답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다. 질문 : 뇌의 어떤 부분이 말을 제어합니까? 답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다. 질문 : 이순신 장군이 전사한 전투는 무슨 전투입니까? 답 : 은 전쟁 중 장군, 장군 전사, 전 장군 세 사람의 이름이 기록되어 있습니다'

In [17]:
get_gpt_output("<s>한국어: 그 도로는 강과 평행으로 뻗어 있다.</s>\
<s>English: The road runs parallel to the river.</s>\
<s>한국어: 그 평행선들은 분기하는 것처럼 보인다.</s>\
<s>English: The parallel lines appear to diverge.</s>\
<s>한국어: 그 도로와 운하는 서로 평행하다.</s>\
<s>English: The road and the canal are parallel to each other.</s>\
<s>한국어: 평행한 은하계라는 개념은 이해하기가 힘들다.</s>\
<s>English: The idea of a parallel universe is hard to grasp.</s>\
<s>한국어: 이러한 전통은 우리 문화에서는 그에 상응하는 것이 없다.</s>\
<s>English: This tradition has no parallel in our culture.</s>\
<s>한국어: 이것은 현대에 들어서는 그 유례를 찾기 힘든 업적이다.</s>\
<s>English: This is an achievement without parallel in modern times.</s>\
<s>한국어: 그들의 경험과 우리 경험 사이에서 유사점을 찾는 것이 가능하다.</s>\
<s>English: It is possible to draw a parallel between their experience and ours.</s>\
<s>한국어: 그 새 학위 과정과 기존의 수료 과정이 동시에 운영될 수도 있을 것이다.</s>\
<s>English: The new degree and the existing certificate courses would run in parallel.</s>\
<s>한국어: 이순신은 조선 중기의 무신이다.</s>\
<s>English: ")

'한국어: 그 도로는 강과 평행으로 뻗어 있다. English: The road runs parallel to the river. 한국어: 그 평행선들은 분기하는 것처럼 보인다. English: The parallel lines appear to diverge. 한국어: 그 도로와 운하는 서로 평행하다. English: The road and the canal are parallel to each other. 한국어: 평행한 은하계라는 개념은 이해하기가 힘들다. English: The idea of a parallel universe is hard to grasp. 한국어: 이러한 전통은 우리 문화에서는 그에 상응하는 것이 없다. English: This tradition has no parallel in our culture. 한국어: 이것은 현대에 들어서는 그 유례를 찾기 힘든 업적이다. English: This is an achievement without parallel in modern times. 한국어: 그들의 경험과 우리 경험 사이에서 유사점을 찾는 것이 가능하다. English: It is possible to draw a parallel between their experience and ours. 한국어: 그 새 학위 과정과 기존의 수료 과정이 동시에 운영될 수도 있을 것이다. English: The new degree and the existing certificate courses would run in parallel. 한국어: 이순신은 조선 중기의 무신이다. English: escaped(잉글시) _The applicated writing._'

In [18]:
get_gpt_output("<s>한국어: 그 도로는 강과 평행으로 뻗어 있다.</s>\
<s>English: The road runs parallel to the river.</s>\
<s>한국어: 그 평행선들은 분기하는 것처럼 보인다.</s>\
<s>English: The parallel lines appear to diverge.</s>\
<s>한국어: 그 도로와 운하는 서로 평행하다.</s>\
<s>English: The road and the canal are parallel to each other.</s>\
<s>한국어: 평행한 은하계라는 개념은 이해하기가 힘들다.</s>\
<s>English: The idea of a parallel universe is hard to grasp.</s>\
<s>한국어: 이러한 전통은 우리 문화에서는 그에 상응하는 것이 없다.</s>\
<s>English: This tradition has no parallel in our culture.</s>\
<s>한국어: 이것은 현대에 들어서는 그 유례를 찾기 힘든 업적이다.</s>\
<s>English: This is an achievement without parallel in modern times.</s>\
<s>한국어: 그들의 경험과 우리 경험 사이에서 유사점을 찾는 것이 가능하다.</s>\
<s>English: It is possible to draw a parallel between their experience and ours.</s>\
<s>한국어: 그 새 학위 과정과 기존의 수료 과정이 동시에 운영될 수도 있을 것이다.</s>\
<s>English: The new degree and the existing certificate courses would run in parallel.</s>\
<s>한국어: 100명은 서 있을 수 있을 것 같은 섬 하나가 넓은 하늘 위에 두둥실 떠 있다.</s>\
<s>English: ")

"한국어: 그 도로는 강과 평행으로 뻗어 있다. English: The road runs parallel to the river. 한국어: 그 평행선들은 분기하는 것처럼 보인다. English: The parallel lines appear to diverge. 한국어: 그 도로와 운하는 서로 평행하다. English: The road and the canal are parallel to each other. 한국어: 평행한 은하계라는 개념은 이해하기가 힘들다. English: The idea of a parallel universe is hard to grasp. 한국어: 이러한 전통은 우리 문화에서는 그에 상응하는 것이 없다. English: This tradition has no parallel in our culture. 한국어: 이것은 현대에 들어서는 그 유례를 찾기 힘든 업적이다. English: This is an achievement without parallel in modern times. 한국어: 그들의 경험과 우리 경험 사이에서 유사점을 찾는 것이 가능하다. English: It is possible to draw a parallel between their experience and ours. 한국어: 그 새 학위 과정과 기존의 수료 과정이 동시에 운영될 수도 있을 것이다. English: The new degree and the existing certificate courses would run in parallel. 한국어: 100명은 서 있을 수 있을 것 같은 섬 하나가 넓은 하늘 위에 두둥실 떠 있다. English: ingeredly Engulish the doesn't responsibly furm of its adm. left since I learned. an operation.ekon Engalish의 영어 표기: shatter: see shockless blanking francissed and add-by remai